In [1]:
import pandas as pd
import sqlite3

# Create a connection object to the SQLite database
database_file_name = "sqlite_db_pythonsqlite.db"
sqlite_connection = sqlite3.connect(database_file_name)

In [6]:
# Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!


question_10_query = """
-- ASSUMPTION: This doesn't include facility's "initialoutlay" and "monthlymaintenance" costs when calculating revenue
WITH cte AS (

    SELECT
        facilities.name AS facility_name,

        -- Since no start/end date is specified in question, we'll use the earliest & latest booking's "start/end date"
        STRFTIME('%Y-%m-%d', MIN(bookings.starttime) OVER()) AS reporting_start_date,
        STRFTIME('%Y-%m-%d', MAX(bookings.starttime) OVER()) AS reporting_end_date,

        -- To get the "per-half-hour" cost and how many "slots" booked
                -- The "IIF" is to differentiate between Guest vs Member Cost (Guest always has ID = 0)
        IIF(bookings.memid = 0, facilities.guestcost, facilities.membercost) AS usd_revenue_per_half_hour_slot,
        bookings.slots AS number_of_slots_booked

    FROM main.Bookings AS bookings

        -- [many:1] Facility
        LEFT JOIN main.Facilities AS facilities
            ON bookings.facid = facilities.facid

        -- [many:1] Member or Guest
        LEFT JOIN main.Members AS members
            ON bookings.memid = members.memid

)
SELECT
    facility_name,
    reporting_start_date,
    reporting_end_date,
    SUM(usd_revenue_per_half_hour_slot * number_of_slots_booked) AS usd_total_revenue

FROM cte

GROUP BY
    facility_name,
    reporting_start_date,
    reporting_end_date
HAVING usd_total_revenue < 1000
;
"""
question_10_df = pd.read_sql(question_10_query, sqlite_connection)
question_10_df

,facility_name,reporting_start_date,reporting_end_date,usd_total_revenue
0,Pool Table,2012-07-03,2012-09-30,270
1,Snooker Table,2012-07-03,2012-09-30,240
2,Table Tennis,2012-07-03,2012-09-30,180


In [7]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

question_11_query = """
SELECT

    -- Member
    members.memid,
    members.surname AS member_surname,
    members.firstname AS memnber_firstname,

    -- Referrer
    member_referrer.memid AS member_referrer_memid,
    (member_referrer.firstname || ' ' || member_referrer.surname) AS member_referrer_first_last_name

FROM main.Members AS members

    -- Get Member who Referred
    LEFT JOIN main.Members AS member_referrer
        ON members.recommendedby = member_referrer.memid

WHERE
    (members.recommendedby IS NOT NULL AND members.recommendedby != '') -- To get members who got referred

ORDER BY
    members.surname,
    members.firstname
;
"""
question_11_df = pd.read_sql(question_11_query, sqlite_connection)
question_11_df

,memid,member_surname,memnber_firstname,member_referrer_memid,member_referrer_first_last_name
0,15,Bader,Florence,9,Ponder Stibbons
1,12,Baker,Anne,9,Ponder Stibbons
2,16,Baker,Timothy,13,Jemima Farrell
3,8,Boothe,Tim,3,Tim Rownam
4,5,Butters,Gerald,1,Darren Smith
5,22,Coplin,Joan,16,Timothy Baker
6,36,Crumpet,Erica,2,Tracy Smith
7,7,Dare,Nancy,4,Janice Joplette
8,20,Genting,Matthew,5,Gerald Butters
9,35,Hunt,John,30,Millicent Purview


In [8]:
# Q12: Find the facilities with their usage by member, but not guests
question_12_query = """
SELECT
    facilities.name AS facility_name,

    -- Since Question didn't specify timeframe, assuming it's from earliest & latest booking date
    STRFTIME('%Y-%m-%d', MIN(bookings.starttime)) AS reporting_start_date,
    STRFTIME('%Y-%m-%d', MAX(bookings.starttime)) AS reporting_end_date,

    SUM(bookings.slots * 0.5) AS facility_usage_total_hours_by_members

FROM main.Bookings AS bookings

    -- [many:1] Facility
    LEFT JOIN main.Facilities AS facilities
        ON bookings.facid = facilities.facid
WHERE
    memid != 0 -- To filter out Guest bookings (memid = 0)

GROUP BY
    facilities.name

ORDER BY
    facility_usage_total_hours_by_members DESC

;
"""
question_12_df = pd.read_sql(question_12_query, sqlite_connection)
question_12_df

,facility_name,reporting_start_date,reporting_end_date,facility_usage_total_hours_by_members
0,Badminton Court,2012-07-05,2012-09-30,543.0
1,Tennis Court 1,2012-07-04,2012-09-30,478.5
2,Massage Room 1,2012-07-03,2012-09-30,442.0
3,Tennis Court 2,2012-07-07,2012-09-30,441.0
4,Snooker Table,2012-07-03,2012-09-30,430.0
5,Pool Table,2012-07-03,2012-09-30,428.0
6,Table Tennis,2012-07-03,2012-09-30,397.0
7,Squash Court,2012-07-04,2012-09-30,209.0
8,Massage Room 2,2012-07-11,2012-09-28,27.0


In [9]:
# Q13: Find the facilities usage by month, but not guests
question_13_query = """
-- ASSUMPTION: Total hours of usage across all facilities
SELECT

    -- Since Question didn't specify timeframe, assuming it's from earliest & latest booking date
    STRFTIME('%m-%Y', bookings.starttime) AS facility_usage_month_year,

    SUM(bookings.slots * 0.5) AS total_hours_usage_by_members

FROM main.Bookings AS bookings

    -- [many:1] Facility
    LEFT JOIN main.Facilities AS facilities
        ON bookings.facid = facilities.facid
WHERE
    memid != 0 -- To filter out Guest bookings (memid = 0)

GROUP BY
    STRFTIME('%m-%Y', bookings.starttime)

ORDER BY
    facility_usage_month_year
;
"""
question_13_df = pd.read_sql(question_13_query, sqlite_connection)
question_13_df

,facility_usage_month_year,total_hours_usage_by_members
0,07-2012,530.5
1,08-2012,1265.5
2,09-2012,1599.5
